In [1]:
import geopandas
import pandas as pd
import numpy as np
import re
import hvplot.pandas #noqa

hvplot.extension('bokeh')
from bokeh.plotting import show

In [2]:
import panel as pn
pn.extension(comms="vscode")

In [3]:
survey_data = pd.read_csv("data/RHoMIS_Full_Data.csv")

/tmp/ipykernel_208/492458180.py:1: DtypeWarning: Columns (9,10,11,13,15,16,19,20,22,23,24,36,39,40,41,43,44,45,46,47,48,49,50,56,57,59,60,61,62,63,65,68,69,71,72,78,80,86,87,89,90,96,98,104,105,107,108,114,116,122,123,125,126,132,134,140,141,143,144,145,150,152,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,219,220,221,224,225,226,227,228,229,230,231,232,233,234,238,240,241,242,243,244,245,249,251,252,264,265,268,270,273,275,277,278,279,282,283,284,288,289,292,293,294,295,296,297,298,299,300,301,302,303,304,306,308,309,313,315,316,317,318,319,320,321,322,323,324,325,326,329,331,333,334,335,338,339,340,343,344,345,348,349,350,351,352,353,354,355,356,357,358,359,360,362,364,365,369,371,372,373,374,375,376,377,378,379,380,381,382,385,387,389,390,391,394,395,396,399,400,401,404,405,406,407,40

This is a massive dataset on 13,30 farm households in 21 countries and 753 indicator variables (made public in 2019).  We'll use `survey_data.columns.values` to understand the kind of indicator variables that are included. Then we will create a subset of the dataset with those that we think are relevant to the agricultural/fertilizer use case study. 

In [4]:
survey_subset = survey_data[['YEAR','ITERATION','SURVEY_ID','GPS_LAT',
 'GPS_LON', 'country', 'local_currency', 'region', 'sublocation',
 'beneficiary', 'participation','respondentsex', 'respondent_is_head',
 'household_position', 'household_type', 'work_away', 'age_malehead',
 'age_femalehead', 'education_level', 'children_under_4', 'children_4to10',
 'males11to24', 'females11to24', 'males25to50','females25to50','malesover50',
 'femalesover50', 'count_people', 'hh_pop_repeat_count', 'land_tenure',
 'landcultivated', 'unitland','areaunits_other', 'landowned',
 'unitland_owned', 'land_ownership', 'areaunits_other_own', 'landrentin',
 'unitland_rentin', 'landrentout', 'unitland_rentout', 'areaunits_other_rent',
 'Farm_labour', 'land_slope', 'homegarden', 'grow_crops', 'crops_all',
 'crops_other1', 'crops_other2','crops_other3', 'farm_planning_who',
 'vegetables_which', 'fruits_which', 'harvest_early', 'early_harvest_crops',
 'early_harvest_reasons', 'early_harvest_reasons_other', 'crops',
 'crop_count','crop_products_yn' ,'crop_products_name',
 'crop_products_other', 'crop_product_consume',
 'crop_products_consume_control', 'crop_product_sell',
 'crop_product_sold_income', 'crop_products_who_control_revenue',
 'Agric_Inputs', 'Agric_Inputs_other', 'fertiliser_crops',
 'fertiliser_amount', 'fertiliser_units', 'fertiliser_type',
 'fertiliser_units_other', 'fertiliser_type_other', 'manure_compost_crops',
 'pesticides_crops','improvedseeds_crops']].copy()
survey_subset

,YEAR,ITERATION,SURVEY_ID,GPS_LAT,GPS_LON,country,local_currency,region,sublocation,beneficiary,...,Agric_Inputs_other,fertiliser_crops,fertiliser_amount,fertiliser_units,fertiliser_type,fertiliser_units_other,fertiliser_type_other,manure_compost_crops,pesticides_crops,improvedseeds_crops
0,2015,1,TZ_CFM_2015,NaN,NaN,Tanzania,NaN,tanga,NaN,NaN,...,NaN,NaN,0,NaN,other,NaN,manure,NaN,NaN,NaN
1,2015,1,TZ_CFM_2015,NaN,NaN,Tanzania,NaN,tanga,NaN,NaN,...,NaN,NaN,75,kg,urea dap,NaN,NaN,NaN,NaN,NaN
2,2015,1,TZ_CFM_2015,NaN,NaN,Tanzania,NaN,tanga,NaN,NaN,...,NaN,NaN,20,kg,urea,NaN,NaN,NaN,NaN,NaN
3,2015,1,TZ_CFM_2015,NaN,NaN,Tanzania,NaN,tanga,NaN,NaN,...,NaN,NaN,15,kg,urea other,NaN,manure,NaN,NaN,NaN
4,2015,1,TZ_CFM_2015,NaN,NaN,Tanzania,NaN,tanga,NaN,NaN,...,NaN,NaN,50,kg,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13305,2018,1,GH_TA7_2018,10.96,-1.17,Ghana,Cedi,KNW,NaN,NaN,...,NaN,maize,3.0,litres,NPK,NaN,NaN,NaN,maize ground_nut,NaN
13306,2018,1,GH_TA7_2018,10.97,-1.16,Ghana,Cedi,KNW,NaN,NaN,...,NaN,millet_late maize rice,3.0,litres,NPK,NaN,NaN,cowpea,NaN,NaN
13307,2018,1,GH_TA7_2018,11.00,-1.01,Ghana,Cedi,KNW,NaN,NaN,...,NaN,maize rice,50.0,kg,NPK,NaN,NaN,maize,cowpea,NaN
13308,2018,1,GH_TA7_2018,11.03,-1.18,Ghana,Cedi,KNM,NaN,NaN,...,NaN,maize,200.0,kg,sulphate_ammonia NPK,NaN,NaN,NaN,maize,NaN


## Data Cleaning
As this is a survey dataset we'll need to do some basic cleaning. 

In [5]:
# Replace empty cells with NaN
survey_subset.replace('', np.nan, inplace=True)

Let us clean up some of the columns we are most interested in like `fertiliser_crops`,`fertiliser_amount`,`fertiliser_units`, `fertiliser_type`, `fertiliser_units_other`,`fertiliser_type_other`, and `manure_compost_crops`. 

In [6]:
# Utils

# replace text with multiple values 
def replace_all(text_, dict_replace):
    for k,v in dict_replace.items():
        text_ = text_.replace(k,v)
    return text_.replace('  ',' ').strip()


# Find words that are always together
def find_unique_words(word_list, is_split=True):
    word_list_unique = {}
    for word in word_list:
        split_word = word.split()
        if not is_split:
            split_word = [word.strip().lower()]
            
        for uniq_word in split_word:
            if not word_list_unique.get(uniq_word):
                word_list_unique[uniq_word] = 0
            word_list_unique[uniq_word] +=1
            
    return dict(sorted(word_list_unique.items(), key=lambda x: x[1]))

# Find words that are always together
def find_bigram_words(word_list):
    bigram_count = defaultdict(int)
    for word in word_list:
        tokens = word.split()
        for i in range(len(tokens) - 1):
            bigram = (tokens[i], tokens[i + 1])
            bigram_count[bigram] += 1
    return bigram_count

# replace string with dict (fix words)
def replace_element(x_val, dict_val, is_split=True):
    if not pd.notnull(x_val):
        return x_val
    x_val = x_val.lower()
    if not is_split:
        return replace_all(x_val, dict_val)
    return " ".join([dict_val.get(i,i)  for i in x_val.split()])
    

### clean fertiliser_crops

In [7]:
# merge crops with others
def merge_cops_other(x_data):
    if not pd.notnull(x_data['fertiliser_crops']):
        return x_data['fertiliser_crops']
    
    fertiliser_crops = str(x_data['fertiliser_crops']).strip().lower()
    for f_crop in list(fertiliser_crops.split()):
        if 'other' in f_crop:
            try:
                fertiliser_crops = replace_all(fertiliser_crops,{f_crop: ''})
                other = x_data[f'crops_{f_crop}']
                if not pd.notnull(other) or other == 'nan':
                    continue
                other = replace_all(str(other).lower(),{' ': '_'})
                fertiliser_crops = replace_all(f'{fertiliser_crops} {other}',{',':' '})
            except:
                pass
    return fertiliser_crops

# convert string
survey_subset['fertiliser_crops'] = survey_subset['fertiliser_crops'].astype('str') 
survey_subset['crops_other1'] = survey_subset['crops_other1'].astype('str') 
survey_subset['crops_other2'] = survey_subset['crops_other2'].astype('str') 
survey_subset['crops_other3'] = survey_subset['crops_other3'].astype('str') 

# merge crops with other crops
survey_subset['fertiliser_crops_all'] = survey_subset.apply(lambda x: merge_cops_other(x), axis=1)

# show merge fertiliser_crops_all (temporal)
survey_merge_tmp = survey_subset[survey_subset['fertiliser_crops'].apply(lambda x: any(['other' in i for i in str(x).split()]))]
survey_merge_tmp[['fertiliser_crops_all','fertiliser_crops','crops_other1','crops_other2','crops_other3']].drop_duplicates(
    subset = ['fertiliser_crops_all'], keep = 'last')

,fertiliser_crops_all,fertiliser_crops,crops_other1,crops_other2,crops_other3
330,pepino,other1,pepino,nan,nan
355,coffee field_beans maize pepino zapallo,coffee field_beans maize other1 other2 other3,pepino,zapallo,nan
1272,cowpea maize millet beans sorghum kales,cowpea maize millet beans sorghum other1 other3,kales,nan,nan
1275,maize beans sorghum kales,maize beans sorghum other3,avocado,pawpaw,kales
1276,maize millet,maize millet other1,nan,nan,nan
...,...,...,...,...,...
12902,teff maize sorghum vegetables fruits tnbaho masho,Teff Maize Sorghum Vegetables Fruits other1 ot...,Tnbaho,masho,nan
12904,teff maize sorghum vegetables masho,Teff Maize Sorghum Vegetables other1,masho,Tnbaho,nan
12905,teff sorghum vegetables fruits masho,Teff Sorghum Vegetables Fruits other1,masho,nan,nan
12917,millet vegetables tenbaho,Millet Vegetables other1,Tenbaho,nan,nan


In [8]:
# first iteration to clean misspelled data or typos
# count crops ( first iteration) 
# fertiliser_crops_unique_initial = survey_subset['fertiliser_crops_all'].tolist()
# unique_words_crops_initial = find_unique_words(fertiliser_crops_unique_initial)
# unique_words_crops_initial

In [9]:
# replace some values with typing and encoding errors manually
fix_words  = {
    'na':'nan',
    'canne_a_sucre':'sugarcane',
    'arroz': 'rice',
    'aubergine': 'eggplant',
    'ayote': 'squash',
    'banano_criollo': 'plantain',
    'citricos': 'citrus_fruits',
    'cow_pea': 'cowpea',
    'culantro_coyote': 'cilantro',
    'flores_tropicales': 'tropical_flowers',
    'choux': 'cabbage',
    'cooton':'cotton',
    'tomate':'tomato',
    'past�que':'watermelon',
    'niébé':'cowpea',
    'palmito': 'heart_of_palm',
    'pomme_de_terre':'potato',
    'patate_douce':'sweet_potato',
    'foddercrop': 'fodder_crop',
    'frijoles': 'beans',
    'ground_nut': 'peanut',
    'ground_nuts': 'peanut',
    'groundnuts': 'peanut',
    'groundnut':'peanut',
    'haricot_bean': 'haricot_beans',
    'kales':'kale',
    'mangoes': 'mango',
    'maíz': 'maize',
    'ma�z': 'maize',
    'onions': 'onion',
    'otras_hortalizas': 'other_vegetables',
    'otro_no.1': 'other1',
    'palma_aceitera': 'oil_palm',
    'patate_douce_et_intore_et_les_avocats':'sweet_potato avocado',
    'peas': 'pea',
    'platóno': 'plantain',
    'plat�no': 'plantain',
    'pimienta':'pepper',
    'rambután': 'rambutan',
    'rambut�n': 'rambutan',
    'sandia': 'watermelon',
    'sorgham': 'sorghum',
    'soy_bean': 'soybean',
    'soya':'soybean',
    'soya_bean': 'soybean',
    'sugar_cane': 'sugarcane',
    'sweetpotato': 'sweet_potato',
    'tef': 'teff',
    'tnbaho': 'tenbaho',
    'yuca':'yucca',
    'yute': 'jute',
    'zapallo':'pumpkin',
    '�ame': 'yam',
    # same or similar meaning
    'green_pepper':'pepper',
    'lentils': 'haricot_beans',
    'cole':'cabbage',
    'dekoko' : 'kale',
    'field_pea': 'pea',
    'meser': 'spinash',
    'oil_nut': 'cashew',
    'pepino': 'cucumber'
    
}
# replace fix words
survey_subset['fertiliser_crops_all_fix'] = survey_subset['fertiliser_crops_all'].apply(lambda x: replace_element(x, fix_words)) 
survey_subset['fertiliser_crops_all_fix'] = survey_subset['fertiliser_crops_all_fix'].replace('nan', np.nan)
fertiliser_crops_unique_final = survey_subset['fertiliser_crops_all_fix'].dropna().tolist()
unique_words_crops_final = find_unique_words(fertiliser_crops_unique_final)
unique_words_crops_final

{'pineapple': 1,
 'guava': 1,
 'carrot': 1,
 'beetroot': 1,
 'oranges': 1,
 'vanilla': 1,
 'passion_fruit': 1,
 'tea': 1,
 'cilantro': 1,
 'macabu': 1,
 'jute': 1,
 'rambutan': 1,
 'tiquisque': 1,
 'tropical_flowers': 1,
 'ibisunzu': 1,
 'intengwa': 1,
 'ubushaza': 1,
 'okra': 1,
 'spinash': 1,
 'radish': 2,
 'cucumber': 2,
 'pumpkin': 2,
 'moong_bean': 2,
 'avocado': 2,
 'squash': 2,
 'tenbaho': 2,
 'soja': 3,
 'coco': 3,
 'citrus_fruits': 3,
 'heart_of_palm': 3,
 'spices': 3,
 'chilipepper': 3,
 'leafy_vegetables': 3,
 'watermelon': 4,
 'other_vegetables': 4,
 'chili': 4,
 'eggplant': 5,
 'oil_palm': 5,
 'garlic': 6,
 'fodder_grass': 6,
 'spinach': 6,
 'masho': 6,
 'roselle_sorrel': 7,
 'guanabana': 7,
 'other1': 7,
 'cabbage': 9,
 'green_grams': 9,
 'pigeon_pea': 9,
 'fonio': 10,
 'mango': 11,
 'yam': 12,
 'cashew': 13,
 'fruits': 13,
 'datil': 13,
 'cacao': 13,
 'millet_late': 14,
 'sweet_potato': 17,
 'papaya': 17,
 'sugarcane': 19,
 'woandzou': 19,
 'broad_bean': 20,
 'pea': 21,


### clean fertiliser_type and fertiliser_type_other

In [10]:
# convert string
survey_subset['fertiliser_type'] = survey_subset['fertiliser_type'].astype('str') 
survey_subset['fertiliser_type_other'] = survey_subset['fertiliser_type_other'].astype('str')

In [11]:
# count fertiliser_type ( first iteration) 
# fertiliser_type_unique_initial = survey_subset['fertiliser_type'].str.lower().dropna().tolist()
# unique_words_ft_initial = find_unique_words(fertiliser_type_unique_initial)
# unique_words_ft_initial

In [12]:
# replace some values with typing and encoding errors manually
fix_words_types  = {
    'na': 'nan',
    'npk16200': 'npk_16-20-0',
    '"npk16200")': 'npk_16-20-0',
    'c("npk16200",': 'npk_16-20-0',
    'c("npk16200': 'npk_16-20-0',
    '"npk16200",': 'npk_16-20-0',
    'dcompound': 'd_compound',
    'c("urea",': 'urea',
    'npk151515': 'npk_15-15-15',
    '"npk151515")': 'npk_15-15-15',
    'npk23_21_0': 'npk_23-21-0',
    'npk1688': 'npk_16-8-8',
    'npk202020': 'npk_20-20-20',
    'npk122412': 'npk_12-24-12',
    'npk103010': 'npk_10-30-10',
    'sulphate_ammonia': 'sulphate_amonium',
    'sulfato_amonio': 'sulphate_amonium',
    'npk_16_8_8' : 'npk_16-8-8',
    'npk_171717': 'npk_17-17-17',
    'npk15331':'npk_15-3-31',
    'npk_171717': 'npk_17-17-17',
    
}
# replace  fix words
survey_subset['fertiliser_type_fix'] = survey_subset['fertiliser_type'].apply(lambda x: replace_element(x, fix_words_types)) 
survey_subset['fertiliser_type_fix'] = survey_subset['fertiliser_type_fix'].replace('nan', np.nan)
# find unique words
fertiliser_type_unique_final = survey_subset['fertiliser_type_fix'].str.lower().dropna().tolist()
unique_words_ft_final = find_unique_words(fertiliser_type_unique_final)
unique_words_ft_final

{'minjingu': 1,
 'lime': 2,
 'blend': 3,
 'fym': 4,
 'phosphate': 4,
 'npk_15-15-15': 16,
 'tsp': 17,
 'compost': 20,
 'npk_16-20-0': 22,
 'npk_17-17-17': 23,
 'ssp': 25,
 'npk_20-20-20': 31,
 'sa': 37,
 'npk1851562': 54,
 'npk_12-24-12': 55,
 'nutran': 76,
 'npk_23-21-0': 106,
 'potassium': 106,
 'npk_16-8-8': 107,
 'phosphorous': 122,
 'npk_10-30-10': 157,
 'npk_15-3-31': 159,
 'sulphate_amonium': 225,
 'd_compound': 299,
 'npk_16': 326,
 'can': 428,
 'other': 557,
 'npk': 726,
 'npk_20': 1218,
 'dap': 1848,
 'urea': 3703}

In [13]:
# count fertiliser_type_other ( first iteration) 
# survey_subset_tmp = survey_subset[survey_subset['fertiliser_type'].apply(lambda x: 'other' in str(x).lower() )]
# fertiliser_type_other_unique_initial = survey_subset_tmp['fertiliser_type_other'].str.lower().dropna().tolist()
# unique_words_fto_initial = find_unique_words(fertiliser_type_other_unique_initial, False)
# unique_words_fto_initial

In [14]:
# replace some values with typing and encoding errors manually

# In this first stage of cleaning, we find that the space works in two ways,
# separating words and separating parts of the same word,
# we do a manual replace to standardize, in the next phase, individual words will be cleaned

fix_words_types_other_1  = {
   # general
    ' y ': ' ',
    ' o ': ' ',
    ';': ' ',
    ' de ': '_de_',
    ',': ' ',
    '.': '-',
    '- ': ' ',
    '—': '-',
    ' and ': ' ',
    '  ': ' ',

    # codes
    '1515': '15-15',
    '15-1515': '15-15-15',
    '18-6-12-': '18-6-12',
    '18-612': '18-6-12',
    '20_20_20': '20-20-20',
    '20 20 0': '20-20-0',
    '16-20-0': '16-20-0',
    '16-20-0y': '16-20-0',
    '18 6 12': '18_6_12',
    '20 20 20': '20-20-20',
    '18 46 0': '18-46-0',
    '17 4 17': '17-4-17',
    '12 24 12': '12-24-12',
    '16 20 20': '16-20-20',
    '20 20': '20-20',
    '15 15': '15-15',
    '46 -0': '46-0',
    '16200': '16-20-0',
    '0 0 60': 'npk_0-0-60',
    '22 3010': '22-30-10',
    '14 18 18': '14-18-18',
    '12 30 10': '12-30-10',
    '20 20 30': '20-20-30',
    '14 23 14': '14-23-14',

    'nutri caf�': 'nutri_cafe',
    'f�rmula': 'formula',
    'formula cafetalera 20': 'formula_cafetalera_20',
    'formula cafetalera 12': 'formula_cafetalera_12',
    'fertilidad caf�': 'fertilidad_cafetalera',
    'formula cafetalera': 'formula_cafetalera',

    'vi sinh': 'vi_sinh',
    'phan vi sinh': 'phan_vi_sinh',
    'phan u vo ca phe': 'phan_u_vo_ca_phe',
    'vo ca phe u': 'vo_ca_phe_u',
    'phan cua duc': 'phan_cua_duc',
    'phan bo': 'phan_bo',
    'phanu men vi': 'phanu_men_vi',
    'phanvi': 'phan_vi',
    'sun phat': 'sun_phat',
    'phan de': 'phan_de',
    'phan chuong': 'phan_chuong',
    'phan ga': 'phan_ga',
    'phi lip pin': 'phi_lip_pin',

    'npk 18': 'npk_18',
    'npk 16': 'npk_16',
    'npk 20': 'npk_20',
    'kpk 12': 'kpk_12',
    'npk 12': 'npk_12',
    'npk 14': 'npk_14',
    'npk 15': 'npk_15',
    'cafetalera 16': 'cafetalera_16',
    'formula 16': 'formula_16',
    'npkn14': 'npk_14',
    'triple 20': '20-20-20',
    'triple 15': '15-15-15',
    # custom
    'green miracle': 'green_miracle',
    'vigor max': 'vigor_max',
    'viga max' : 'viga_max',
    'biological fertilizer': 'biological_fertilizer',
    "don't know the type he used": "dont_know",
    "don't know" : "dont_know",
    'sulphate of amonium (s/a)':'sulphate_amonium',
    'super green': 'super_green',
    'rapid grow': 'rapid_grow',
    'super grow': 'super_grow',
    '6 litros': '6_litros',
    'yara nila': 'yaramila',
    'yara yara mila': 'yaramila',
    'yara winner': 'yarawinner',
    'ne sait pas': "dont_know",
    'npk maïs npk coton': 'npk_maize npk_coton',
    'npk maïs et npk coton': 'npk_maize npk_coton',
    'npk maïs': 'npk_maize',
    'ferti cafe': 'ferti_cafe',
    'musrurd cake': 'musrurd_cake',
    'mosterdoil cake': 'mosterd_oil_cake',
    'mosterd oil cake':'mosterd_oil_cake',
    
}
# replace  fix words
survey_subset['fertiliser_type_other_fix'] = survey_subset['fertiliser_type_other'].apply(
    lambda x: replace_element(x, fix_words_types_other_1, False)) 
survey_subset['fertiliser_type_other_fix'] = survey_subset['fertiliser_type_other_fix'].replace('na', np.nan)
survey_subset['fertiliser_type_other_fix'] = survey_subset['fertiliser_type_other_fix'].replace('nan', np.nan)

# find unique words 
survey_subset_tmp = survey_subset[survey_subset['fertiliser_type'].apply(lambda x: 'other' in str(x).lower() )]
fertiliser_type_other_unique_final = survey_subset_tmp['fertiliser_type_other_fix'].dropna().tolist()
unique_words_fto_final = find_unique_words(fertiliser_type_other_unique_final, False)
unique_words_fto_final


{'vid mix': 1,
 '20-20-0 15-15-15': 1,
 '16-20-0 18-6-12': 1,
 '20-20-0 18-6-12': 1,
 '16-20-0 sulfato': 1,
 'sulfato_de_amonio 20-20-0': 1,
 '15-15-15 sulfato': 1,
 '16-15 3-15': 1,
 '38-5n 7-2s': 1,
 'formula_16-20-0 sulfato': 1,
 'formula sulfato': 1,
 'sulfato formula_16-20-0': 1,
 'kcl 12-24-12 18_6_12': 1,
 '12-24-12 nutricafe 20-20 18-46-0-': 1,
 'nitrato 20-20 18_6_12': 1,
 'nitrato kcl 20-20 18-46-0': 1,
 '20-20 17-4-17': 1,
 '20-20 12-24-12': 1,
 '20-20 nutricafe': 1,
 '12-24-12 20-20': 1,
 '20-20-20 nutri_cafe': 1,
 'kcl 18-46-0 12-24-12 sulfato_de_amonio': 1,
 'formula_cafetalera_20-20-20': 1,
 'formula_cafetalera_12-24-12 nitrato_de_amonio': 1,
 'fertilidad_cafetalera formula_cafetalera_16-20-20': 1,
 'calcio mapa magnesio': 1,
 '18-46-0 12-24-12': 1,
 'fastrac 20-20 18-46-0 map nitrato_de_c�lcio nitrato_de_amonio': 1,
 'nitr�geno f�sforo potacio': 1,
 '17-4-17': 1,
 'calcium': 1,
 'biofertilizer': 1,
 'musrurd_cake ashes': 1,
 'zink': 1,
 'calcium zink thymet': 1,
 'biofe

In [15]:
# Merge fertiliser_type with others (fertiliser_type_other)

def merge_fertiliser_type_other(x_data):
    if not pd.notnull(x_data['fertiliser_type_fix']):
        return x_data['fertiliser_type_fix']
    
    fertiliser_type = str(x_data['fertiliser_type_fix']).strip().lower()
    for f_type in list(fertiliser_type.split()):
        if 'other' == f_type:
            try:
                fertiliser_type =  replace_all(fertiliser_type, {'other': ''})
                other = x_data['fertiliser_type_other_fix']
                if not pd.notnull(other):
                    continue
                fertiliser_type = replace_all(f'{fertiliser_type} {other}',{})
            except:
                pass
    return fertiliser_type

# merge fertiliser_type_all with other 
survey_subset['fertiliser_type_all'] = survey_subset.apply(lambda x: merge_fertiliser_type_other(x), axis=1)
# show merge fertiliser_type_all
survey_merge_tmp = survey_subset[survey_subset['fertiliser_type'].apply(lambda x: 'other' in str(x))]
survey_merge_tmp[['fertiliser_type_all','fertiliser_type_fix','fertiliser_type_other_fix']].drop_duplicates(
        subset = ['fertiliser_type_fix'], keep = 'last')



,fertiliser_type_all,fertiliser_type_fix,fertiliser_type_other_fix
126,dap minjingu manure,dap minjingu other,manure
329,npk_16-8-8 sulfato formula_16-20-0,npk_16-8-8 other,sulfato formula_16-20-0
343,urea npk_16-8-8 formula_cafetalera_20-20-20,urea npk_16-8-8 other,formula_cafetalera_20-20-20
1797,urea dap ssp tsp zink,urea dap ssp tsp other,zink
1865,urea dap ssp zinc,urea dap ssp other,zinc
1880,urea npk dap ssp zinc,urea npk dap ssp other,zinc
1917,urea npk dap ssp tsp mosterd_oil_cake,urea npk dap ssp tsp other,mosterd_oil_cake
2565,urea npk_20-20-20 vi_sinh,urea npk_20-20-20 other,vi_sinh
2597,npk potassium sa,npk potassium other,sa
2606,urea phosphorous sa,urea phosphorous other,sa


In [16]:
# find unique words 
# fertiliser_type_merge_unique_initial = survey_subset['fertiliser_type_all'].dropna().tolist()
# unique_words_ftm_initial = find_unique_words(fertiliser_type_merge_unique_initial)
# unique_words_ftm_initial

In [17]:
# replace some values with typing and encoding errors manually

# In this second stage of cleaning, we look for unique words

fix_words_types_merge  = {
     **fix_words_types,
    '18-6-12-':'npk_18-6-12',
    '18-46-0-':'dap_18-46-0',
    'formula_cafetalera_20-20-20': 'npk_20-20-20',
    'formula_cafetalera_12-24-12': 'npk_12-24-12',
    'calcio': 'calcium',
    'magnesio': 'magnesium',
    'nitrato_de_c�lcio': 'calcium_nitrate',
    'nitr�geno': 'nitrogen',
    'f�sforo': 'phosphorus',
    'potacio': 'potassium',
    'biofertilozer': 'biofertilizer',
    'biofrertilizet': 'biofertilizer',
    'phan_bo': 'cover_fertilizer',
    'sulphate_of_amonium_(s/a)': 'sulphate_amonium',
    'vigor_max-': 'vigor_max',
    'booster(foliar)': 'booster_foliar',
    'biological_fertilizer': 'biofertilizer',
    'formula_16-20-0': 'npk_16-20-0',
    'formula': 'npk',
    '18_6_12': 'npk_18-6-12',
    'nitrato': 'nitrate',
    '17-4-17': 'npk_17-4-17',
    'nitrato_de_amonio': 'ammonium_nitrate',
    'zink': 'zinc',
    '20-20-20': 'npk_20-20-20',
    'vo_ca_phe_u': 'wet_coffee_pulp',
    '16-20': 'npk_16-20-0',
    'phan_u_vo_ca_phe': 'coffee_pulp_fertilizer',
    '18-6-12': 'npk_18-6-12',
    '12-24-12': 'npk_12-24-12',
    '20-20-0': 'npk_20-20-0',
    '20-20': 'npk_20-20-0',
    'sulfato_de_amonio': 'sulphate_amonium',
    '16-20-0': 'npk_16-20-0',
    'vi_sinh': "microorganisms",
    'phan_chuong': 'cage_fertilizer',
    'sulfato': 'sulphate',
    'urea': 'npk_46_0_0',
    'npk_18-24': 'npk_18-24-0',
    # 
    '15-15-15': 'npk_15-15-15',
    '18-46-0' : 'dap_18-46-0',
    'dap' : 'dap_18-46-0',
    'npk_20': 'npk_20-20-20',
    'npk_16': 'npk_16-16-16',
    'can': 'calcium_ammonium_nitrate',
    'sa': 'sulphate_amonium',
    'ammonium_sulphate': 'sulphate_amonium',
    'formula_cafetalera_16-20-20': 'npk_16-20-20',
    'mapa': 'monoammonium phosphate',
    'map': 'monoammonium phosphate'
  
}
# replace  fix words
survey_subset['fertiliser_type_all_fix'] = survey_subset['fertiliser_type_all'].apply(
    lambda x: replace_element(x, fix_words_types_merge)) 
survey_subset['fertiliser_type_all_fix'] = survey_subset['fertiliser_type_all_fix'].replace('none', np.nan)

# find unique words 
fertiliser_type_merge_unique_final = survey_subset['fertiliser_type_all_fix'].dropna().tolist()
unique_words_ftm_final = find_unique_words(fertiliser_type_merge_unique_final)
unique_words_ftm_final


{'vid': 1,
 'mix': 1,
 '16-15': 1,
 '3-15': 1,
 '38-5n': 1,
 '7-2s': 1,
 'nutri_cafe': 1,
 'fertilidad_cafetalera': 1,
 'npk_16-20-20': 1,
 'magnesium': 1,
 'fastrac': 1,
 'calcium_nitrate': 1,
 'nitrogen': 1,
 'phosphorus': 1,
 'musrurd_cake': 1,
 'ashes': 1,
 'thymet': 1,
 'enzyme': 1,
 'ជីបីពឞ���': 1,
 'ជីប���តង': 1,
 'ជីក���បាលក���របី': 1,
 'ជួយប���ប���នឬ��': 1,
 'liquid': 1,
 'ជីធម���មជាតិ': 1,
 '0sa': 1,
 'phan_cua_duc': 1,
 'cover_fertilizer': 1,
 'phanu_men_vi_sinh': 1,
 'phan_vi_sinh': 1,
 'voi': 1,
 'isa': 1,
 'phan_de': 1,
 'phi_lip_pin': 1,
 'mua': 1,
 'vo': 1,
 'caphe': 1,
 'u': 1,
 '<u+0e9a><u+0ecd><u+0ec8><u+0ec4><u+0e94><u+0ec9><u+0ec3><u+0e8a><u+0ec9>': 1,
 'rock': 1,
 'yard': 1,
 'waste': 1,
 'folier': 1,
 'yarawinner': 1,
 'npk_18-24-0': 1,
 'organique': 1,
 'vigor_max': 1,
 'superglue': 1,
 'dudu': 1,
 'theonex': 1,
 'booster_foliar': 1,
 'vegimax': 1,
 'viga_max': 1,
 'liters': 1,
 'micro': 1,
 'food': 1,
 'etg': 1,
 'foliar': 1,
 'feed': 1,
 'azufrada': 1,
 'cal':

In [26]:
# show example data
survey_merge_tmp = survey_subset[survey_subset['fertiliser_type_all_fix'].apply(lambda x: 'dap' in str(x))]
survey_merge_tmp[['fertiliser_type_all_fix','fertiliser_type_fix','fertiliser_type','fertiliser_type_other_fix','fertiliser_type_other']].drop_duplicates(
        subset = ['fertiliser_type_all_fix'], keep = 'last')

,fertiliser_type_all_fix,fertiliser_type_fix,fertiliser_type,fertiliser_type_other_fix,fertiliser_type_other
41,npk_46_0_0 npk dap_18-46-0 manure,urea npk dap other,urea npk dap other,manure,manure
44,npk_46_0_0 dap_18-46-0 vid mix,urea dap other,urea dap other,vid mix,vid mix
126,dap_18-46-0 minjingu manure,dap minjingu other,dap minjingu other,manure,manure
138,dap_18-46-0 manure,dap other,dap other,manure,manure
146,npk_46_0_0 dap_18-46-0 manure,urea dap other,urea dap other,manure,manure
...,...,...,...,...,...
12574,npk_46_0_0 dap_18-46-0 compost,urea dap other,urea DAP other,compost,Compost
12577,npk_46_0_0 dap_18-46-0 manures,urea dap other,urea DAP other,manures,Manures
12736,npk_46_0_0 dap_18-46-0 biofertilizer,urea dap other,urea DAP other,biological_fertilizer,Biological fertilizer
12791,dap_18-46-0,dap,DAP,NaN,nan


## Using `hvplot.explorer` for inspiration of further exploratory data analysis

We have to revert back to the survey subset as `geodataframes` are not yet supported. Using this the `hvplot.explorer` option we can perform some exploratory data analysis by examining relationships between indicators. Once you've identified something of interest you can export the code to develop that graphic using `hvexplorer.plot_code()` as shown in the example below. 

In [27]:
hvexplorer = hvplot.explorer(survey_subset)
display(hvexplorer)

BokehModel(combine_events=True, render_bundle={'docs_json': {'2b909737-cd92-4cf5-91ac-247a7e920216': {'defs': …

In [21]:
# hvexplorer.plot_code()

Interacting with the `hvplot.explorer` GUI we created a scatterplot examining fertilizer types used by country and exported the code below:

In [22]:
graph1 = survey_subset.hvplot(groupby=['YEAR'], kind='scatter', x='country', y=['fertiliser_type_all_fix'])

In [23]:
display(graph1.opts(xrotation=45))

BokehModel(combine_events=True, render_bundle={'docs_json': {'4d8abac3-cb2c-4c41-bb23-86ef7639c37b': {'defs': …

Further cleaning/grouping of the fertilizer types will be required to make this graph fully readable - but this is a start. Could potentially use Levenshtein’s distance to group together similar survey entries. 

In [24]:
# gdf['country'].unique()

In [25]:
# gdf_subset = gdf[["fertiliser_amount","country","fertiliser_type","geometry"]].dropna()

In [ ]:
# import plotly.express as px
# gdf_subset = px.choropleth(
#     gdf,
#     locations="country",
#     color="fertiliser_type",
#     hover_name="country",
#     color_continuous_scale=px.colors.sequential.Plasma,
#     animation_frame="YEAR",
#     animation_group="country",
#     range_color=[0, 100000],
# )

# # display graph in browser (new window opens)
# # a bokeh server is automatically started
# bokeh_server = pn.Row(gdf_subset).show()

## Examing the survey's spatial distribution
We'll then convert it to a `geospatial pandas dataframe` to make it easier to project on a map. Keep in mind, however, that we are still examining only a `subset` of the full survey dataset. As there are `753` indicator variables, we have only focused a subset of the dataset with those indicators of greatest interest to us in the food/agriculture/fertilizer study space. 

In [ ]:
gdf = geopandas.GeoDataFrame(
    survey_subset, geometry=geopandas.points_from_xy(survey_subset.GPS_LON, survey_subset.GPS_LAT), crs="EPSG:4326"
)
gdf

In [ ]:
gdf.hvplot.points('GPS_LON', 'GPS_LAT', geo=True, color='red', alpha=0.05, tiles='ESRI')

## Examining temporal distribution and examination of sampling bias in the dataset

In the map above we can see that the survey dataset is mostly centered around countries in Central America, sub-Saharan Africa, and Asia. Now, we'll look at the temporal distribution of survey responses within the dataset. Examining the graph below we see that the majority of countries surveyed have only been done once between 2015-2018, with the exception of Tanzania (4 years), Burkina Faso and Ethiopia (3 years), and Mali, Kenya, and Ghana (2 years). 

In [ ]:
gdf.hvplot.scatter(x='country', y='YEAR', rot=45, width=1000)

Looking a step further at the total number of survey responses (rows) for each country, we see that Burkina Faso (3507/13310 = 22.96%) and Tanzania (2508/13310 = 18.84%) make up a large chunk of the survey responses in the subset that we've examined. Therefore roughly 40% of the responses in this subset are derived from two countries. 

In [ ]:
gdf["country"].value_counts().hvplot.bar(invert=True, flip_yaxis=True, height=500)